<img width="8%" alt="LangChain.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LangChain.png" style="border-radius: 15%">

# LangChain - Create Agent

**Tags:** #langchain #neo4j #graph #knowledge

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2024-03-25 (Created: 2024-03-25)

**Description:** This notebook creates an Agent.

**References:**
- [Medium - Using a Knowledge Graph to Implement a DevOps RAG Application](https://medium.com/neo4j/using-a-knowledge-graph-to-implement-a-devops-rag-application-b6ba24831b16)

## Input

### Import libraries

In [1]:
import naas
import os
import requests
try:
    from langchain.graphs import Neo4jGraph
    from langchain.vectorstores.neo4j_vector import Neo4jVector
    from langchain.chains import RetrievalQA, GraphCypherQAChain
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType
except:
    !pip install langchain==0.1.13 --user
    from langchain.graphs import Neo4jGraph
    from langchain.vectorstores.neo4j_vector import Neo4jVector
    from langchain.chains import RetrievalQA, GraphCypherQAChain
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType
try:
    from langchain_openai import OpenAIEmbeddings
    from langchain_openai import ChatOpenAI
except:
    !pip install langchain-openai=0.1.1 --user
    from langchain_openai import OpenAIEmbeddings
    from langchain_openai import ChatOpenAI
try:
    import neo4j
except:
    !pip install neo4j==5.18.0 --user
    import neo4j
from neo4j import GraphDatabase
import pandas as pd

### Setup variables

In [2]:
url = "neo4j+s://a23aa9f6.databases.neo4j.io:7687"
username = naas.secret.get("NEO4J_USERNAME")
password = naas.secret.get("NEO4J_PASSWORD")
graph_source_url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/e90b0c9386bf8be15b199e8ac8f83fc265a2ac57/microservices.json"
os.environ['OPENAI_API_KEY'] = naas.secret.get("OPENAI_API_KEY")

## Model

### Neo4j Graph Setup

In [3]:
graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)
graph

### Get graph dataset

In [4]:
import_query = requests.get(graph_source_url).json().get('query')
import_query

"MERGE (catalog:Microservice {name: 'CatalogService', technology: 'Java'}) MERGE (order:Microservice {name: 'OrderService', technology: 'Python'}) MERGE (user:Microservice {name: 'UserService', technology: 'Go'}) MERGE (payment:Microservice {name: 'PaymentService', technology: 'Node.js'}) MERGE (inventory:Microservice {name: 'InventoryService', technology: 'Java'}) MERGE (shipping:Microservice {name: 'ShippingService', technology: 'Python'}) MERGE (review:Microservice {name: 'ReviewService', technology: 'Go'}) MERGE (recommendation:Microservice {name: 'RecommendationService', technology: 'Node.js'}) MERGE (auth:Microservice {name: 'AuthService', technology: 'Node.js'}) MERGE (db:Microservice {name: 'Database', technology: 'SQL'}) MERGE (cache:Microservice {name: 'Cache', technology: 'In-memory'}) MERGE (mq:Microservice {name: 'MessageQueue', technology: 'Pub-Sub'}) MERGE (api:Microservice {name: 'ExternalAPI', technology: 'REST'}) MERGE (bugFixCatalog:Task {name: 'BugFix', description:

### Import graph into Neo4j

In [5]:
graph.query(
    import_query
)

[]

## Output

### Vector Index & Retrieval QA

In [6]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)

In [7]:
vector_qa.invoke(
    "How will recommendation service be updated?"
)

{'query': 'How will recommendation service be updated?',
 'result': 'The RecommendationService will be updated by adding a new feature to provide more personalized and accurate product recommendations to users. This update will leverage user behavior and preference data to enhance the recommendation algorithm. The current status of the update is "in progress."'}

In [8]:
vector_qa.invoke(
    "How many open tickets there are?"
)

{'query': 'How many open tickets there are?',
 'result': 'There are 4 open tickets currently.'}

### GraphCypherQAChain

In [9]:
graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [10]:
cypher_chain.invoke(
    "How will recommendation service be updated?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice {name: 'recommendation service'})-[:MAINTAINED_BY]->(t:Team)
RETURN t.name as TeamName
Full Context:
[]

> Finished chain.


{'query': 'How will recommendation service be updated?',
 'result': "I don't know the answer."}

In [11]:
cypher_chain.invoke(
    "How many open tickets there are?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task {status: 'open'}) RETURN COUNT(t)
Full Context:
[{'COUNT(t)': 5}]

> Finished chain.


{'query': 'How many open tickets there are?',
 'result': 'There are 5 open tickets.'}

### Knowledge Graph Agent

In [12]:
tools = [
    Tool(
        name="Tasks",
        func=vector_qa.invoke,
        description="""Useful when you need to answer questions about descriptions.
        Not useful for counting.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.invoke,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of 
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

mrkl = initialize_agent(
    tools, 
    ChatOpenAI(temperature=0, model_name='gpt-4'),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

/home/ftp/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [13]:
response = mrkl.invoke("How will recommendation service be updated?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Tasks` with `How will recommendation service be updated?`


{'query': 'How will recommendation service be updated?', 'result': 'The RecommendationService is currently undergoing an update to add a new feature that will provide more personalized and accurate product recommendations to users. This update involves leveraging user behavior and preference data to enhance the recommendations. The status of this update is "in progress."'}The RecommendationService is currently undergoing an update to add a new feature that will provide more personalized and accurate product recommendations to users. This update involves leveraging user behavior and preference data to enhance the recommendations. The status of this update is "in progress."

> Finished chain.
{'input': 'How will recommendation service be updated?', 'output': 'The RecommendationService is currently undergoing an update to add a new feature that will provide more personalized and

In [14]:
response = mrkl.invoke("How many open tickets there are?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `How many open tickets there are?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task {status: 'open'}) RETURN COUNT(t)
Full Context:
[{'COUNT(t)': 5}]

> Finished chain.
{'query': 'How many open tickets there are?', 'result': 'There are 5 open tickets.'}There are 5 open tickets.

> Finished chain.
{'input': 'How many open tickets there are?', 'output': 'There are 5 open tickets.'}


In [15]:
response = mrkl.invoke("What is the latest post about data product?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Tasks` with `What is the latest post about data product?`


{'query': 'What is the latest post about data product?', 'result': 'The latest update related to data products is the "Update" task for InventoryService, which is currently in progress. This update aims to include real-time stock updates to ensure accurate inventory levels.'}The latest update related to data products is the "Update" task for InventoryService, which is currently in progress. This update aims to include real-time stock updates to ensure accurate inventory levels.

> Finished chain.
{'input': 'What is the latest post about data product?', 'output': 'The latest update related to data products is the "Update" task for InventoryService, which is currently in progress. This update aims to include real-time stock updates to ensure accurate inventory levels.'}


In [16]:
response = mrkl.invoke("What is the name of the teams?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `What is the name of the teams?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Team) RETURN t.name
Full Context:
[{'t.name': 'TeamA'}, {'t.name': 'TeamB'}, {'t.name': 'TeamC'}, {'t.name': 'TeamD'}]

> Finished chain.
{'query': 'What is the name of the teams?', 'result': 'TeamA, TeamB, TeamC, TeamD'}The names of the teams are TeamA, TeamB, TeamC, and TeamD.

> Finished chain.
{'input': 'What is the name of the teams?', 'output': 'The names of the teams are TeamA, TeamB, TeamC, and TeamD.'}


In [17]:
response = mrkl.invoke("What is Alice's team?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `What is Alice's team?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Alice'})-[:PART_OF]->(t:Team) RETURN t.name
Full Context:
[{'t.name': 'TeamA'}]

> Finished chain.
{'query': "What is Alice's team?", 'result': "Alice's team is TeamA."}Alice is a part of TeamA.

> Finished chain.
{'input': "What is Alice's team?", 'output': 'Alice is a part of TeamA.'}


In [18]:
response = mrkl.invoke("What is Frank's team?")
print(response)



> Entering new AgentExecutor chain...

Invoking: `Graph` with `What is Frank's team?`




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Frank'})-[:PART_OF]->(t:Team) RETURN t.name
Full Context:
[{'t.name': 'TeamB'}]

> Finished chain.
{'query': "What is Frank's team?", 'result': "Frank's team is TeamB."}Frank is a part of TeamB.

> Finished chain.
{'input': "What is Frank's team?", 'output': 'Frank is a part of TeamB.'}
